In [130]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

from settings.key import GPT_API_KEY
openai.api_key = GPT_API_KEY
from LLMSearch.AutoDataset.GPTQuestions import gpt_function_call

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [131]:

from LLMSearch.AnswerBot import AnswerBot,parse_answer
from settings.key import GPT_API_KEY, DEEPL_API_KEY
from LLMSearch.GPTQuery import GPTQuery
from LLMSearch.BM25DB import BM25DB
from LLMSearch.SQLTextDB import SQLTextDB
from LLMSearch.Embedding.SBERTFineTuneEmbedding import SBERTFineTuneEmbedding
from LLMSearch.VectorSearcher import VectorSearcher

embedder=SBERTFineTuneEmbedding()
searcher=VectorSearcher(embedder,chunk_size_limit=500)


#bm=BM25DB(initiate=True)
#searcher=SQLTextDB()

# initiate bot module
bot1 = AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=searcher,
                DEEPL_API_KEY=DEEPL_API_KEY,
                )

[nltk_data] Downloading package punkt to /home/kh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmplq527ht7
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmplq527ht7/_remote_module_non_scriptable.py
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [188]:
from LLMSearch.DocSplitter import split_text,remove_stopwords
path=bot1.searcher.path_list[-1]
path_list=list(set(bot1.searcher.path_list))
path_list=[path for path in path_list if path.find("koko")==-1]


In [192]:
path_list=path_list[:5]

In [193]:

import shutil
count=0
for path in path_list:
    with open(path, 'r') as f:
        text = f.read()
    chunk_list = split_text(text, bot1.searcher.chunk_size_limit)

    # remove stopwords
    chunk_list = [remove_stopwords(chunk) for chunk in chunk_list]


    for text in chunk_list:
        questions=gpt_function_call(text)

        out_path="UserData/meta/qa/qa.txt"
        for q in questions:
            with open(out_path, 'a') as f:
                f.write(path)
                f.write("|||||")
                f.write(questions[q])
                f.write("|||||")
                f.write(text)
                f.write("\n")
        print(questions)
    #コピーを作成してバックアップ
    shutil.copyfile(out_path, f"UserData/meta/qa/qa_backup{count}.txt")
    count+=1

{'q1': 'この研究の目的は何ですか？', 'q2': 'どのような方法でポリアミンの検知を試みましたか？', 'q3': 'どのような現象が観察されましたか？', 'q4': 'c-AuNPsはどのような特性を持っていますか？', 'q5': 'スペルミンに対してc-AuNPsはどのような反応を示しましたか？', 'q6': '他のポリアミンと比べてスペルミンへの反応はどうだったのですか？', 'q7': 'c-AuNPsの検知感度はどのくらいですか？', 'q8': '反応時間はどれくらいでしたか？'}
{'q1': 'c-AuNPsの反応性が高まる条件は何か？', 'q2': 'ポリアミンの検知におけるc-AuNPsの粒径の重要性は何か？', 'q3': '実尿中のポリアミンを検知するためにc-AuNPsを使用した結果，どのような変化が見られたか？', 'q4': '金ナノ粒子の特徴的な性質は何か？', 'q5': 'AuNPsの水溶液の色はどのようなものか？', 'q6': '粒子表面状態の化学的変化により何が起こるか？', 'q7': '凝集が生じると可視吸収スペクトルはどのように変化するか？', 'q8': 'c-AuNPsを使った尿中ポリアミンの検知可能性の確認はなぜ重要か？'}
{'q1': 'AuNPsの特殊比色測定機能は何に利用されることができますか？', 'q2': 'AuNPsの特殊比色測定機能を利用して、何が検知可能ですか？', 'q3': 'AuNPsはどのような特徴を持っていますか？', 'q4': 'AuNPsの合成法の一つで広く利用されているものは何ですか？', 'q5': 'AuNPsの調製に利用されるクエン酸還元剤の特徴は何ですか？', 'q6': 'ポリアミンとは何ですか？', 'q7': 'ポリアミンにはどのようなアミノ基が結合していますか？', 'q8': 'ポリアミンはどのような化学分野で利用されていますか？'}
{'q1': 'ポリアミンはどのような生理活性物質として働くことが知られていますか？', 'q2': 'ポリアミンはどのような組織や細胞に広く分布していますか？', 'q3': 'ポリアミンは何に対して強い親和力をもつことが知られていますか？', 'q4': 'ポリアミンとがんの関係についてはどのような研究

'UserData/meta/qa/qa_backup.txt'